In [1]:
import pandas as pd
import numpy as np
import opendatasets as od
import time
import datetime
from torchsummary import summary
import torch.nn as nn

import torch
from sklearn.model_selection import train_test_split

In [2]:
dataset_url = 'https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction'
od.download(dataset_url)

Skipping, found downloaded files in "./new-york-city-taxi-fare-prediction" (use force=True to force download)


In [4]:
df = pd.read_csv('./NYCTaxiFares_small.csv')
df['pickup_datetime'].max()

'2010-04-25 03:59:42 UTC'

In [5]:
## Running this cell is really fucking slow, but it works.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

def havardsine_distance(lat1, long1, lat2, long2):
    r = 6371

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)

    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(long2 - long1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = (r * c) # convert to kilometer
    return d

df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

airport_cords = [
    [40.691547, -74.180202], # NEWARK
    [40.773281, -73.869845], # LAGUARDIA
]

# When I wrote this code, only God and I knew what was going on. Today, only God knows.
def close_to_airport(values):
    pickup_latitude = values['pickup_latitude']
    pickup_longitude = values['pickup_longitude']
    dropoff_latitude = values['dropoff_latitude']
    dropoff_longitude = values['dropoff_longitude']

    # JFK is special, it needs a bigger radius
    jfkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.645042, -73.786928) <= 1.00
    jfkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.645042, -73.786928) <= 1.00

    # Newark is a small airport but their parking log is BIG.
    newarkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.691547, -74.180202) <= 1.00
    newarkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.691547, -74.180202) <= 1.00

    # Third largest, it's an abstract airport with a stretched out U shaped parking place
    # LaGuardia
    laguardiapickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.773855, -73.871712) <= 0.50
    laguardiadropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.773855, -73.871712) <= 0.50

    return 1 if jfkpickup or jfkdropoff or newarkpickup or newarkdropoff or laguardiapickup or laguardiadropoff else 0

df['is_by_airport'] = df.apply(close_to_airport, axis=1)

In [6]:
my_time = df['pickup_datetime'][0]

In [7]:
df['EDTdate'] = df['pickup_datetime'] - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour'] < 12, 0, 1)

In [8]:
df['Weekday'] = pd.to_numeric(df['EDTdate'].dt.strftime("%w")) # an int between 0 - 6, representing the weekdays
df['Month'] = pd.to_numeric(df['EDTdate'].dt.strftime("%m")) # an int between 0 - 6, representing the weekdays

#Formatting this correctly, we can use the dates to get the historical weather data for the dates
df['FullDate'] = df['EDTdate'].dt.strftime('%x')
#print(df['FullDate'].min())
#print(df['FullDate'].max())
print(df['FullDate'].unique())

['04/19/10' '04/17/10' '04/11/10' '04/16/10' '04/22/10' '04/23/10'
 '04/15/10' '04/20/10' '04/21/10' '04/14/10' '04/13/10' '04/12/10'
 '04/24/10' '04/18/10']


In [399]:
'''
api_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"
new_york_middle_lat = ""
new_york_middle_long = ""
unique = np.sort(df['FullDate'].unique())
start = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())
end = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())
url = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"

print(url)
'''
# Historical weather data, not working

'\napi_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"\nnew_york_middle_lat = ""\nnew_york_middle_long = ""\nunique = np.sort(df[\'FullDate\'].unique())\nstart = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())\nend = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())\nurl = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"\n\nprint(url)\n'

In [9]:
# lets have some weather-y fuuuuuuuuuuuun-fuuuuuun-function
# see this for doc https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
weather_data = pd.read_csv('nyc-weather-data.csv')
weather_labels = ['SNWD', 'SNOW', 'AWND'] #snow depth, did it snow
for index, item in enumerate(weather_labels):
    weather_data.loc[weather_data[item] <= -9999, item] = 0 # -9999 means that there is no data

# df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

weather_data['DATE'] = weather_data.apply(lambda row: datetime.datetime.strptime(str(row.DATE), '%Y%m%d').strftime('%m/%d/%y'), axis=1)
weather_data = weather_data[['DATE','PRCP','SNWD','SNOW','TMAX','TMIN','AWND']]

weather_data['TMIN'] = (weather_data['TMIN'] / 10) # the API returns degrees in an old format. We therefore convert the value to a tenth of its own form. Check the docu for this.
weather_data['TMAX'] = (weather_data['TMAX'] / 10)
weather_data['PRCP'] = weather_data['PRCP'] / 10

weather_data['AWND'] = weather_data['AWND'] / 2.237 # meters per second

df.rename(columns={'FullDate':'DATE'}, inplace=True)

merged_df = pd.merge(df, weather_data, how="left",on="DATE")

In [10]:
network_data = merged_df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km', 'passenger_count', 'is_by_airport', 'Hour', 'AMorPM', 'Weekday', 'Month', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN', 'AWND', 'fare_amount']]

# normalize the data

print(network_data)

prediction = network_data['fare_amount']

network_data.drop('fare_amount', axis=1)
X = network_data.values

y = prediction.values.reshape(-1,1)

        pickup_latitude  pickup_longitude  dropoff_latitude  \
0             40.730521        -73.992365         40.744746   
1             40.740558        -73.990078         40.744114   
2             40.751118        -73.994149         40.766235   
3             40.756422        -73.990485         40.748192   
4             40.734202        -73.990976         40.743115   
...                 ...               ...               ...   
119995        40.784590        -73.955857         40.736789   
119996        40.772727        -73.996329         40.740413   
119997        40.749772        -73.988574         40.707799   
119998        40.724529        -74.004449         40.730765   
119999        40.771920        -73.955415         40.763015   

        dropoff_longitude   dist_km  passenger_count  is_by_airport  Hour  \
0              -73.975499  2.126312                1              0     4   
1              -73.974232  1.392307                1              0    11   
2           

In [11]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

In [12]:
X = torch.tensor(train_X, dtype=torch.float)
y = torch.tensor(train_y, dtype=torch.float)
test_X_tensor = torch.Tensor(test_X)
test_y_tensor = torch.Tensor(test_y)

train_ds = torch.utils.data.TensorDataset(X, y)
test_ds = torch.utils.data.TensorDataset(test_X_tensor, test_y_tensor)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)

In [13]:
class NYCTaxiPredictor(nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(18, 42),
            nn.ReLU(),
            nn.Linear(42, 42),
            nn.ReLU(),
            nn.Linear(42, 42),
            nn.ReLU(),
            nn.Linear(42, 1),
        )

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = NYCTaxiPredictor()
summary(model, tuple([18]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 42]             798
              ReLU-2                   [-1, 42]               0
            Linear-3                   [-1, 42]           1,806
              ReLU-4                   [-1, 42]               0
            Linear-5                   [-1, 42]           1,806
              ReLU-6                   [-1, 42]               0
            Linear-7                    [-1, 1]              43
Total params: 4,453
Trainable params: 4,453
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epochs in range(500):
    for data in train_dl:
        X, y = data
        optimizer.zero_grad()
        output = model(X)
        print(output)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(loss.data)

tensor([[-0.5278],
        [-0.4285],
        [-0.4533],
        [-0.5464],
        [-0.4071],
        [-0.4426],
        [-0.3679],
        [-0.5130],
        [-0.4598],
        [-0.4224],
        [-0.4531],
        [-0.4040],
        [-0.5082],
        [-0.5056],
        [-0.4760],
        [-0.3685],
        [-0.4777],
        [-0.4828],
        [-0.4645],
        [-0.4233],
        [-0.5258],
        [-0.4251],
        [-0.4573],
        [-0.5005],
        [-0.4281],
        [-0.4687],
        [-0.4647],
        [-0.4275],
        [-0.4710],
        [-0.4609],
        [-0.4978],
        [-0.5346],
        [-0.4387],
        [-0.4233],
        [-0.4454],
        [-0.3852],
        [-0.3850],
        [-0.4406],
        [-0.4474],
        [-0.4595],
        [-0.3715],
        [-0.3816],
        [-0.4748],
        [-0.4704],
        [-0.4188],
        [-0.4876],
        [-0.5136],
        [-0.4241],
        [-0.3950],
        [-0.5962],
        [-0.4520],
        [-0.3873],
        [-0.

KeyboardInterrupt: 